In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread("../data/mva2023_sod4bird_train/images/00001.jpg", cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
import torch

In [ ]:
data = torch.tensor(img).unsqueeze(0).float()
data.shape

In [ ]:
data = data.permute(0, 3, 1, 2)
data.shape

In [ ]:
import swin_transformer as st
from importlib import reload
reload(st)
from swin_transformer import SwinTransformer

In [ ]:
model = SwinTransformer(
    img_size=2016,
    patch_size=4,
    in_chans=3,
    num_classes=1,
    embed_dim=96,
    depths=[2, 2, 6, 2],
    num_heads=[3, 6, 12, 24],
    window_size=7,
    mlp_ratio=4.0,
    qkv_bias=True,
    qk_scale=None,
    drop_rate=0.0,
    attn_drop_rate=0.0,
    drop_path_rate=0.0,
    norm_layer=torch.nn.LayerNorm,
    ape=False,
    patch_norm=True,
    use_checkpoint=False,
    fused_window_process=False,
)
model

In [ ]:
data.shape

In [ ]:
img = cv2.resize(img, (2016, 2016))
data = torch.tensor(img).unsqueeze(0).float()
data = data.permute(0, 3, 1, 2)
data.shape

In [ ]:
model(data).shape

In [ ]:
import swin_T as swin
reload(swin)
from swin_T import swin_t

In [ ]:
model = swin_t()
model

In [ ]:
pred = model(data)
pred.shape

In [1]:
from neck import Neck
from backbone import Backbone

import torch
import torch.nn as nn

In [2]:
temp = torch.randn(1, 3, 1024, 1024)

In [3]:
backbone_model = Backbone(hid_dim=96, layers=(2, 2, 2, 2), heads=(3, 6, 12, 24))

/home/schecter/miniconda3/envs/pikachu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
backbone_out, feature_maps = backbone_model(temp)

In [5]:
backbone_out.shape

torch.Size([1, 768, 32, 32])

In [6]:
neck_model = Neck(hid_dim=96, layers=(2, 2, 2, 2), heads= (3, 6, 12, 64), channels=768, feature_maps=feature_maps)

In [7]:
for i in range(4):
    print(feature_maps[i].shape)

torch.Size([1, 96, 256, 256])
torch.Size([1, 192, 128, 128])
torch.Size([1, 384, 64, 64])
torch.Size([1, 768, 32, 32])


In [8]:
neck_out = neck_model(backbone_out.permute(0, 2, 3, 1))

In [9]:
neck_out.shape

torch.Size([1, 256, 256, 96])